# nanoHUB Nanoplasticity

### Ben Haley (HUBzero) and David Elbert (JHU)

In [1]:
import sys, json, time
from urllib import urlencode
from urllib2 import urlopen, Request, HTTPError
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook

## Both nanoHUB user account and web app registered with nanoHUB are required 
### Go to https://nanohub.org/members/myaccount to create an account
### Go to https://nanohub.org/developer/api/applications/new to create an application


Add web app and user data here:

In [3]:
import sys
sys.path.append('/davidelbert/persistent/NanoHUB_API_Example/NanoHUB_remote')

In [4]:
import NanoHUB_remote.mysecrets

In [6]:
import NanoHUB_remote

#
# This design is strictly for convenience so that this notebook can be 
# shared without storing any secrets.
from NanoHUB_remote.mysecrets import auth_data

# Authenticate; use headers in all subsequent steps
headers = NanoHUB_remote.authenticate(auth_data)

In [7]:
#class NanoHUBApiSettings:
api_url = r'https://nanohub.org/api' # URL for the nanoHUB api
tools_path = r'tools/list'
oauth_path = r'developer/oauth/token'
tools_status_path = r'tools/status'
tools_result_path = r'tools/output'
tools_run_path = r'tools/run'
    
  
sleep_time = 1.5 # Time to wait for NanoHUB database updates


In [ ]:
# Generate the XML driver to run the tool with our inputs
#driver_json = NanoHUB_remote.get_driver('lammpstool', tool_inputs, headers)

## Utility Functions (skip below to "Start Here!")
No actual tinkering needed here

In [9]:
def wait_for_processing():
    time.sleep(NanoHUBApiSettings.sleep_time)

In [10]:
def do_get(path, data):
    """Send a GET to url/path; return JSON output"""
    d = urlencode(data)
    r = Request('{0}/{1}?{2}'.format(NanoHUBApiSettings.api_url, path, d) , data=None, headers=NanoHUBApiSettings.headers)
    try:
        u = urlopen(r)
    except HTTPError as e:
        msg = 'GET {0} failed ({1}): {2}\n'.format(r.get_full_url(), e.code, e.reason)
        sys.stderr.write(msg)
        sys.exit(1)
    return json.loads(u.read())

In [11]:
 def do_post(path, data):
    """Send a POST to url/path; return JSON output"""
    d = urlencode(data)
    r = Request('{0}/{1}'.format(NanoHUBApiSettings.api_url, path) , data=d, headers=NanoHUBApiSettings.headers)
    try:
        u = urlopen(r)
    except HTTPError as e:
        msg = 'POST {0} failed ({1}): {2}\n'.format(r.get_full_url(), e.code, e.reason)
        sys.stderr.write(msg)
        sys.exit(1)
    return json.loads(u.read())

In [12]:
def authenticate():
    """Authenticate against """
    auth_json = do_post(NanoHUBApiSettings.oauth_path, NanoHUBApiSettings.auth_data)
    sys.stdout.write('Authenticated\n')

    NanoHUBApiSettings.headers = {
       'Authorization': 'Bearer {}'.format(auth_json['access_token'])
    }


In [13]:
def get_tools():
    """Returns a list of tools"""
    
    tools = do_get(NanoHUBApiSettings.tools_path, {})
    return tools

In [14]:
def tool_status(session_id):
    """Get the tool status of the run"""
    status_data = {
        'session_num' : session_id
    }
    time.sleep(NanoHUBApiSettings.sleep_time)
    status = do_get(NanoHUBApiSettings.tools_status_path, status_data)
    return status

In [15]:
def tool_result(session_id, run_file):
    """Pull in the result of the tool run"""
    result_data = {
       'session_num': session_id,
       'run_file': run_file
    }
    result_json = do_get(NanoHUBApiSettings.tools_result_path, result_data)
    return result_json

In [16]:
def run_tool(tool_name, driver_xml):
    """Runs the tool and blocks until a result is provided"""
    
    run_data = {
        'app': tool_name,
        'xml': driver_xml
    }
    
    run_json = do_post(NanoHUBApiSettings.tools_run_path, run_data)
    session_id = run_json['session']
    sys.stdout.write('Started job (session {})\n'.format(session_id))
    
    status = tool_status(session_id)
    while not status['finished']:
        wait_for_processing()
        status = tool_status(session_id)
    
    # Sleep to allow for success record to be created
    wait_for_processing()
        
    run_file = status['run_file']
    sys.stdout.write('Run finished')
    return tool_result(session_id, run_file)

## Start Here!
### Step 1. Authenticate first by running "authenticate()"

In [17]:
authenticate()

NameError: global name 'NanoHUBApiSettings' is not defined

### Step 2A. Look for a Tool

In [19]:
# Retrieve all tools (with short description and version)
print get_tools()

NameError: global name 'NanoHUBApiSettings' is not defined

### Step 2B. Use a Tool
  1. Specify a tool name
  2. Specificy the payload to send to the tool (e.g. the path to a driver xml file)
  3. Run the tool with the "run_tool" function

In [20]:
# Pick a tool
tool_name = 'nanoplasticity' # Specify the tool short-name (from tool url)

# Here we specify the payload for the tool, in this example, I am reading from a file,
# Though, the payload can be specified as a string as well
### Assuming the example driver is in the current path
with open('nanoplasticity_driver_xml', 'r') as f: # Specify the payload for the tool
    driver_str = f.read()
xml_result = run_tool(tool_name, driver_str) # Return results

IOError: [Errno 2] No such file or directory: 'nanoplasticity_driver_xml'

### Step 3. Analyze results
#### Quick and dirty parsing of the xml to get all curves

In [ ]:
fig = plt.figure()
ax0 = fig.add_subplot(111)

### Find and plot all XY curves from output
### In this case all curves share the same XY variables (not always the case)
out = xml_result['output']

### All 2D curves use "xy" xml tag
outsplit = out.split('<xy>')
for sp, curvetxt in enumerate(outsplit[1:]):
    curve = curvetxt.split('</xy>')[0]
    with open('xy'+str(sp)+'.txt','w') as f:
        f.write(curve)
    npcurve = np.atleast_2d(np.loadtxt('xy'+str(sp)+'.txt'))
    ax0.plot(npcurve[:,0], npcurve[:,1])

### Find axis labels and units
name = ['']*2
for a, direction in enumerate(['xaxis','yaxis']):
    axis = out.split('<'+direction+'>')[1].split('</'+direction+'>')[0]
    for tag in ['label', 'units']:
        try:
            print name[a]
            name[a] += axis.split('<'+tag+'>')[1].split('</'+tag+'>')[0] + ' '
        except IndexError:
            print('No {} for {}'.format(tag, direction))
ax0.set_xlabel(name[0])
ax0.set_ylabel(name[1])
plt.show()

### number (single value), text, and other output available

## Possible issues:
1. Authentication errors
    1. You don't have a nanoHUB account (see above)
    2. You don't have a nanoHUB web app (see above)
2. Authentication works, but the run generates authorization error
    1. You already have the maximum (3) sessions running (check your nanoHUB homepage)